# EXTRACCIÓN DE ARTÍCULOS Y ENSAYOS DE INMUNOTERAPIA PERIOPERATORIA EN EL CÁNCER DE PULMÓN

## CLINICALTRIALS.GOV

Extracción de ensayor clínicos relacionados

In [ ]:
# librerias
import requests
import pandas as pd

# URL
BASE_URL = "https://clinicaltrials.gov/api/v2/studies"

# Búsqueda
queries = ["Non-Small Cell Lung Cancer surgery",
           "Non-Small Cell Lung Cancer neoadjuvant",
           "perioperative immunotherapy lung cancer",
           "neoadjuvant immunotherapy lung cancer",
           "checkpoint inhibitor lung cancer surgery"
           ]

all_studies = {}

for query in queries:
    params = {"query.term": query, "pageSize": 100, "format": "json"}

    resp = requests.get(BASE_URL, params=params)
    data = resp.json()

    for s in data.get("studies", []):
        nct = s["protocolSection"]["identificationModule"]["nctId"]
        all_studies[nct] = s   # evita duplicados

print("Total estudios combinados:", len(all_studies))
studies = list(all_studies.values())

df = pd.DataFrame(all_studies.values())
# se guarda el dataset bruto
df.to_csv("clinicaltrials_raw.csv", index=False)
df.head()

Total estudios combinados: 325


,protocolSection,derivedSection,hasResults,resultsSection,documentSection
0,{'identificationModule': {'nctId': 'NCT0637908...,{'miscInfoModule': {'versionHolder': '2026-01-...,False,NaN,NaN
1,{'identificationModule': {'nctId': 'NCT0256642...,{'miscInfoModule': {'versionHolder': '2026-01-...,False,NaN,NaN
2,{'identificationModule': {'nctId': 'NCT0677953...,{'miscInfoModule': {'versionHolder': '2026-01-...,False,NaN,NaN
3,{'identificationModule': {'nctId': 'NCT0499367...,{'miscInfoModule': {'versionHolder': '2026-01-...,True,{'participantFlowModule': {'preAssignmentDetai...,{'largeDocumentModule': {'largeDocs': [{'typeA...
4,{'identificationModule': {'nctId': 'NCT0634803...,{'miscInfoModule': {'versionHolder': '2026-01-...,False,NaN,NaN


In [ ]:
import pandas as pd
import numpy as np
import re

# PARTIMOS DE studies = lista de JSONs

# EXTRACCIÓN DE CAMPOS DEL JSON v2


def extract_trial(study):
  """
  Extrae los campos principales de un ensayo clínico a partir de la
  estructura JSON de ClinicalTrials.gov.

  Parameters
  ----------
  study : dict
      Diccionario con la información completa del estudio.

  Returns
  -------
  dict
      Diccionario con identificador, título, condiciones, estado,
      fase, fechas, intervenciones, resumen y localizaciones.
  """
  ps = study.get("protocolSection", {})
  ident = ps.get("identificationModule", {})
  status = ps.get("statusModule", {})
  cond = ps.get("conditionsModule", {})
  design = ps.get("designModule", {})
  arms = ps.get("armsInterventionsModule", {})
  desc = ps.get("descriptionModule", {})
  locs = ps.get("contactsLocationsModule", {})

  return {"NCTId": ident.get("nctId"), "title": ident.get("briefTitle"),
          "conditions": cond.get("conditions"),
          "status": status.get("overallStatus"),
          "phase": ", ".join(design.get("phases", []))
          if design.get("phases") else None,
          "start_date": status.get("startDateStruct", {}).get("date"),
          "completion_date": status.get("completionDateStruct", {}).get("date"),
          "interventions_raw": arms.get("interventions"),
          "summary": desc.get("briefSummary"),
          "locations_raw": locs.get("locations"),}


df_raw = pd.DataFrame([extract_trial(s) for s in studies])
print("Campos extraídos:", df_raw.columns.tolist())


Campos extraídos: ['NCTId', 'title', 'conditions', 'status', 'phase', 'start_date', 'completion_date', 'interventions_raw', 'summary', 'locations_raw']


In [ ]:
# LIMPIEZA DE INTERVENCIONES (IO, CHEMO, CIRUGÍA)

def extract_interventions(interventions):
  """
  Extrae nombres y tipos de intervención en minúsculas.
  """
  if not isinstance(interventions, list):
    return None
  names = []
  for iv in interventions:
      name = iv.get("name")
      itype = iv.get("type")
      if name:
          names.append(name.lower())
      if itype:
          names.append(itype.lower())
  return ", ".join(set(names)) if names else None


df_raw["interventions"] = df_raw["interventions_raw"].apply(extract_interventions)


# ---------- FLAGS CLÍNICAS IMPORTANTES (reducidas a 1 función) ----------

def flag_terms(row, cols_text=(), cols_list=(), terms=()):
  """
  Construye un texto normalizado (minúsculas) a partir de columnas del row y
  devuelve True si algún término aparece como substring.
  """
  text_parts = []

  for col in cols_text:
      v = row.get(col)
      if isinstance(v, str):
          text_parts.append(v.lower())

  for col in cols_list:
      v = row.get(col)
      if isinstance(v, list):
          text_parts.append(" ".join(str(x).lower() for x in v))

  text = " ".join(text_parts)
  return any(t in text for t in terms)


IO_TERMS = ["pd-1","pd1","pd-l1","pdl1","ctla-4", "pembrolizumab","nivolumab",
            "atezolizumab", "durvalumab","tislelizumab","envafolimab",
            "sacituzumab","immunotherapy","checkpoint","ici"]

SURG_TERMS = ["surgery","surgical","resection","lobectomy", "segmentectomy",
              "pneumonectomy","thoracotomy", "vats","rats","curative", "operable",
              "resectable"]

PERI_TERMS = ["neoadjuvant","preoperative","perioperative"]

NSCLC_TERMS = ["nsclc", "non-small cell", "non small cell", "lung cancer",
               "resectable lung"]


df_raw["is_IO"] = df_raw.apply(
    lambda r: flag_terms(r, cols_text=("interventions", "summary"),
                         terms=IO_TERMS),
    axis=1
)

df_raw["is_surgery"] = df_raw.apply(
    lambda r: flag_terms(r, cols_text=("interventions", "summary", "title"),
                         terms=SURG_TERMS),
    axis=1
)

df_raw["is_peri"] = df_raw.apply(
    lambda r: flag_terms(r, cols_text=("summary", "title"), terms=PERI_TERMS),
    axis=1
)

df_raw["is_lung"] = df_raw.apply(
    lambda r: flag_terms(r, cols_text=("summary", "title"), cols_list=(
        "conditions",), terms=NSCLC_TERMS),
    axis=1
)


# ---------- LIMPIEZA DE PAÍSES, LOCALIZACIONES Y CONDICIONES ----------

def extract_country(locations):
  """
  Extrae países únicos en minúsculas desde localizaciones.
  """
  if not isinstance(locations, list):
      return None
  countries = [loc.get("country", "") for loc in locations if loc.get("country")]
  countries = [c.lower().strip() for c in countries if c]
  return list(set(countries)) if countries else None


df_raw["countries_raw"] = df_raw["locations_raw"].apply(extract_country)


def normalize_country(c):
  """
  Normaliza un nombre de país.
  """
  if c is None:
      return None
  s = str(c).lower().strip()
  s = s.replace(".", " ").replace(",", " ")
  s = " ".join(s.split())
  return s


COUNTRY_MAP = {"china": "China", "p r china": "China", "pr china": "China",
               "people's republic of china": "China", "usa": "United States",
               "united states": "United States",
               "united states of america": "United States",
               "korea": "South Korea",
               "republic of korea": "South Korea",
               "south korea": "South Korea", "italy": "Italy", "japan": "Japan",
               "spain": "Spain", "germany": "Germany", "france": "France",
               "switzerland": "Switzerland", "australia": "Australia",
               "canada": "Canada", "belgium": "Belgium", "turkey": "Turkey",
               "austria": "Austria", "the netherlands": "Netherlands",
               "netherlands": "Netherlands"}


def unify_country_list(country_list):
  """
  Unifica países usando un mapa de correspondencias.
  """
  if not isinstance(country_list, list):
     return None
  clean = []
  for c in country_list:
      c_norm = normalize_country(c)
      if c_norm in COUNTRY_MAP:
          clean.append(COUNTRY_MAP[c_norm])
      else:
          clean.append(c_norm.title())
  return list(set(clean))


def flatten_unique_sorted(x):
  """
  Convierte una lista en string único y ordenado.
  """
  if not isinstance(x, list) or len(x) == 0:
      return None
  clean = [str(i).strip() for i in x]
  return ", ".join(sorted(set(clean)))


df_raw["countries"] = df_raw["countries_raw"].apply(unify_country_list)
df_raw["countries"] = df_raw["countries"].apply(flatten_unique_sorted)

df_raw["conditions"] = df_raw["conditions"].apply(flatten_unique_sorted)


# ---------- FILTRO CLÍNICO FINAL AMPLIADO ----------

df_clean = df_raw[
    df_raw["is_IO"] &
    df_raw["is_lung"] &
    (df_raw["is_surgery"] | df_raw["is_peri"] |
     df_raw["summary"].str.contains("resect", case=False, na=False))
].copy()

print("Ensayos relevantes finales:", len(df_clean))


# ---------- VARIABLES FINALES Y EXPORTACIÓN ----------

final_cols = ["NCTId","title","status","phase", "start_date","completion_date",
              "conditions","interventions", "is_IO","is_surgery","is_peri",
              "countries"]

df_final_trials = df_clean[final_cols].copy()

df_final_trials.to_csv("clinicaltrials_clean_final.csv", index=False,
                       encoding="utf-8-sig")

print("CSV exportado: clinicaltrials_clean_final.csv")

with pd.ExcelWriter("clinicaltrials_clean_final.xlsx", engine="openpyxl") as writer:
    df_final_trials.to_excel(writer, sheet_name="ClinicalTrials_clean", index=False)
    sheet = writer.sheets["ClinicalTrials_clean"]
    for idx, col in enumerate(df_final_trials.columns, start=1):
      max_len = max(df_final_trials[col].astype(str).apply(len).max(), len(col)) + 2
      sheet.column_dimensions[chr(64+idx)].width = min(max_len, 50)

print("Excel exportado: clinicaltrials_clean_final.xlsx")


Ensayos relevantes finales: 141
CSV exportado: clinicaltrials_clean_final.csv
Excel exportado: clinicaltrials_clean_final.xlsx


In [ ]:
df_final_trials.head()

,NCTId,title,status,phase,start_date,completion_date,conditions,interventions,is_IO,is_surgery,is_peri,countries
0,NCT06379087,Radiation Therapy Followed by Tislelizumab and...,RECRUITING,PHASE2,2024-05-01,2026-12-31,Non-Small Cell Lung Cancer,"anlotinib hydrochloride capsule, hypofractiona...",True,False,True,China
1,NCT02566421,Genomic Sequencing in Determining Treatment in...,TERMINATED,NA,2015-10,2017-08,"Metastatic Neoplasm, Recurrent Neoplasm, Recur...","laboratory biomarker analysis, targeted therap...",True,True,False,United States
3,NCT04993677,A Study of SEA-CD40 Given With Other Drugs in ...,COMPLETED,PHASE2,2021-10-06,2024-11-25,"Carcinoma, Non-Small- Cell Lung, Melanoma","sea-cd40, drug, carboplatin, pemetrexed, pembr...",True,True,False,"Canada, France, Germany, Spain, Sweden, United..."
13,NCT04205552,Neoadjuvant Nivolumab Combination Treatment in...,RECRUITING,PHASE2,2020-03-04,2025-06,"NSCLC Stage II, NSCLC, Stage I, NSCLC, Stage IIIA","nivolumab 10 mg/ml intravenous solution, relat...",True,True,True,"Belgium, Germany, Netherlands"
15,NCT06385262,TOP 2301: Neoadjuvant Chemo for NSCLC,RECRUITING,PHASE2,2025-03-17,2029-10-30,Non Small Cell Lung Cancer,"chemotherapy, cemiplimab, drug, alirocumab",True,True,True,United States


------
## Extracción de datos PUBMED

In [ ]:
!pip install biopython

from Bio import Entrez, Medline
import pandas as pd
import time

# Obligatorio para la API
Entrez.email = "kdeaguiar@uoc.edu"

query = """(lung cancer[Title/Abstract] OR NSCLC) AND (neoadjuvant OR preoperative
OR perioperative) AND (immunotherapy OR checkpoint inhibitor OR PD-1 OR PD-L1 OR
CTLA-4)"""

# 1. Buscar PMIDs
search_handle = Entrez.esearch(db="pubmed", term=query, retmax=20000)
search_results = Entrez.read(search_handle)
search_handle.close()

pmids = search_results["IdList"]
len(pmids)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 24.5 MB/s eta 0:00:00


1559

Los registros se descargaron mediante la API Entrez de NCBI en formato MEDLINE, en bloques de 200 identificadores PubMed (PMID), respetando las políticas de uso mediante pausas entre peticiones.

In [ ]:
from Bio import Entrez, Medline
def fetch_details(id_list):
  """
  Descarga registros completos de PubMed en formato MEDLINE.
  """
  ids = ",".join(id_list)
  handle = Entrez.efetch(db="pubmed", id=ids, rettype="medline", retmode="text")
  records = handle.read()
  handle.close()
  return records


# ---------- DESCARGA EN BLOQUES ----------
all_records = []
block_size = 200

for start in range(0, len(pmids), block_size):
    end = min(start + block_size, len(pmids))
    block_ids = pmids[start:end]

    records = fetch_details(block_ids)
    all_records.append(records)
    time.sleep(1)  # para no saturar la API

# ---------- GUARDADO MEDLINE ----------
with open("pubmed_raw.txt", "w") as f:
    f.write("".join(all_records))


# ---------- PARSEO A DATAFRAME ----------
with open("pubmed_raw.txt") as f:
    records = list(Medline.parse(f))

pubmed_df = pd.DataFrame(records)
pubmed_df.head()

In [ ]:
import pandas as pd
import numpy as np
import re

# PARTIMOS DE pubmed_df YA CARGADO

df = pubmed_df.copy()
print("Artículos brutos:", len(df))


# ASEGURAR COLUMNAS Y SELECCIONAR CAMPOS ÚTILES

needed_cols = ["PMID", "TI", "AB", "JT", "DP", "AU", "PT", "OT", "AD", "AID"]
for col in needed_cols:
    if col not in df.columns:
        df[col] = np.nan

df = df[needed_cols].copy()

# NORMALIZAR TEXTO Y AÑO

for c in ["TI", "AB", "JT", "PT", "OT", "AD", "AID"]:
    df[c] = df[c].astype(str)

# Año desde DP
df["year"] = df["DP"].astype(str).str.extract(r"(\d{4})")
df["year"] = pd.to_numeric(df["year"], errors="coerce")

# Cortamos en 2005 (inicio de IO)
df = df[df["year"] >= 2005]
print("Tras filtrar año ≥2005:", len(df))


# CLASIFICAR TIPO DE PUBLICACIÓN Y EXCLUIR LETTER/EDITORIAL/CONFERENCE

def classify_pub_type(pt):
  """
  Clasifica el tipo de publicación según el campo Publication Type.
  """
  s = str(pt).lower()
  if "clinical trial" in s:
      return "clinical_trial"
  if "journal article" in s:
      return "article"
  if "review" in s:
      return "review"
  if "letter" in s:
      return "letter"
  if "editorial" in s:
      return "editorial"
  if "case report" in s:
      return "case_report"
  if "conference" in s or "congress" in s or "abstract" in s:
      return "conference"
  return "other"


df["pub_type"] = df["PT"].apply(classify_pub_type)
df = df[~df["pub_type"].isin(["letter", "editorial", "conference", "other"])]
print("Tras filtrar tipo de publicación:", len(df))

# Quitar journals que contengan "conference"
df["JT_lower"] = df["JT"].str.lower()
df = df[~df["JT_lower"].str.contains("conference", na=False)]
print("Tras excluir 'conference' en journal:", len(df))


# LIMPIEZA DE AFILIACIÓN (INSTITUCIÓN Y PAÍS)

# Diccionarios para unificar países
COUNTRY_MAP = {
    "usa": "United States",
    "united states": "United States",
    "u.s.a": "United States",
    "u.s.a.": "United States",
    "united states of america": "United States",
    # China
    "china": "China",
    "p r china": "China",
    "pr china": "China",
    "p r of china": "China",
    "people's republic of china": "China",
    "p.r. china": "China",
    "p. r. china": "China",
    "P.R. china": "China",
    "P. R. china": "China",
    # South Korea
    "korea": "South Korea",
    "republic of korea": "South Korea",
    "south korea": "South Korea",

    "deutschland": "Germany",
    "germany": "Germany",
    "espana": "Spain",
    "spain": "Spain",
    "uk": "United Kingdom",
    "england": "United Kingdom",
    "scotland": "United Kingdom",
    "wales": "United Kingdom",
    "united kingdom": "United Kingdom",
    "italy": "Italy",
    "japan": "Japan",
    "canada": "Canada",
    "france": "France",
    "switzerland": "Switzerland",
    "australia": "Australia",
    "belgium": "Belgium",
    "turkey": "Turkey",
}

US_STATES = {"alabama","alaska","arizona","arkansas","california","colorado",
             "connecticut", "delaware","florida","georgia","hawaii","idaho",
             "illinois","indiana","iowa","kansas","kentucky","louisiana","maine",
             "maryland","massachusetts","michigan","minnesota","mississippi",
             "missouri","montana","nebraska","nevada","new hampshire",
             "new jersey","new mexico","new york","north carolina",
             "north dakota","ohio","oklahoma","oregon","pennsylvania",
             "rhode island","south carolina","south dakota","tennessee","texas",
             "utah","vermont","virginia","washington","west virginia",
             "wisconsin","wyoming","pa","ca","ny","tx","ma","md"}


def parse_affiliation(aff):
  """
  Extrae institución + país de la afiliación MEDLINE completa.
  """
  if pd.isna(aff) or aff.lower() == "nan":
      return None, None

  s = aff.strip()
  s = s.strip("[]'\"")

  # Cortar multiple affiliation
  s = s.split(";")[0]

  # Quitar "Electronic address"
  s = re.sub(r"electronic address:.*", "", s, flags=re.I)

  parts = [p.strip(" .'\"") for p in s.split(",") if p.strip()]
  if not parts:
      return None, None

  institution = parts[0].strip()
  last = parts[-1].strip().lower()

  # Normalizar país
  if last in COUNTRY_MAP:
      country = COUNTRY_MAP[last]
  elif last in US_STATES:
      country = "United States"
  else:
      # si parece extraño o demasiado corto, ignoramos
      country = last.title() if len(last) > 3 else None

  return institution, country


df["institution"], df["country"] = zip(*df["AD"].apply(parse_affiliation))


# FILTRO CLÍNICO FINO (PULMÓN + IO + NEOD/ PERIOP + CIRUGÍA)

lung_terms = ["lung", "nsclc", "non-small cell"]
io_terms = [
    "immunotherapy","checkpoint","pd-1","pd1","pd-l1","pdl1","ctla-4",
    "pembrolizumab","nivolumab","atezolizumab","durvalumab","tislelizumab",
    "sacituzumab"]
peri_terms = ["neoadjuvant","preoperative","perioperative"]
surg_terms = ["surgery","surgical","resection","resectable","lobectomy",
              "segmentectomy","pneumonectomy","thoracotomy","vats","rats"]

df["TI_lower"] = df["TI"].str.lower()
df["AB_lower"] = df["AB"].str.lower()

lung_mask = df["TI_lower"].str.contains("|".join(lung_terms), na=False) | df[
    "AB_lower"].str.contains("|".join(lung_terms), na=False)
io_mask   = df["TI_lower"].str.contains("|".join(io_terms),   na=False) | df[
    "AB_lower"].str.contains("|".join(io_terms),   na=False)
peri_mask = df["TI_lower"].str.contains("|".join(peri_terms), na=False) | df[
    "AB_lower"].str.contains("|".join(peri_terms), na=False)
surg_mask = df["TI_lower"].str.contains("|".join(surg_terms), na=False) | df[
    "AB_lower"].str.contains("|".join(surg_terms), na=False)

df_clean = df[lung_mask & io_mask & (peri_mask | surg_mask)].copy()
print("Artículos tras filtro clínico estricto:", len(df_clean))


# EXTRAER DOI Y GENERAR URL

doi_regex = re.compile(r"(10\.\d{4,9}/\S+)", re.IGNORECASE)


def extract_doi(aid):
  """
  Extrae el DOI a partir del campo Article Identifier.
  """
  if pd.isna(aid):
      return None
  s = str(aid)
  m = doi_regex.search(s)
  if m:
      return m.group(1).rstrip("].")
  return None


df_clean["DOI"] = df_clean["AID"].apply(extract_doi)
df_clean["url"] = "https://pubmed.ncbi.nlm.nih.gov/" + df_clean[
    "PMID"].astype(str) + "/"


# 7LIMPIEZA DE AUTORES (AU) Y OT (KEYWORDS)


def clean_authors(a):
  """
  Limpia columna AU independientemente de si está en formato:
  - lista Python real
  - string tipo "['Autor1', 'Autor2']"
  - string plano
  - NaN
  """
  if a is None:
      return None

  # Caso 1: lista real
  if isinstance(a, list):
      return ", ".join([str(x).strip() for x in a])

  # Caso 2: string que representa una lista
  s = str(a)

  # Si la cadena contiene '[' y ']', intentamos parsear
  if s.startswith("[") and s.endswith("]"):
      # quitar corchetes
      s = s.strip("[]")
      # eliminar comillas
      s = s.replace("'", "").replace('"', "")
      # separar por coma
      parts = [p.strip() for p in s.split(",") if p.strip()]
      return ", ".join(parts)

  # Caso 3: texto normal
  return s.strip()


def clean_keywords(ot):
  """
  Limpia y normaliza la lista de palabras clave.
  """
  if pd.isna(ot):
      return None
  s = str(ot).strip("[]")
  s = s.replace("'", "").replace('"', "")
  s = " ".join(s.split())
  return s


df_clean["AU"] = df_clean["AU"].apply(clean_authors)
df_clean["OT"] = df_clean["OT"].apply(clean_keywords)

# COLUMNAS FINALES Y EXPORTACIÓN

final_cols = [
    "PMID","TI","AB","JT","year","pub_type",
    "AU","OT","institution","country","DOI","url"]

df_final = df_clean[final_cols].copy()
print("Filas finales:", len(df_final))

# CSV
df_final.to_csv("pubmed_clean_final.csv", index=False, encoding="utf-8-sig")
print("- CSV exportado:", "pubmed_clean_final.csv")


# EXCEL
output_excel = "pubmed_clean_final.xlsx"
with pd.ExcelWriter(output_excel, engine="openpyxl") as writer:
    df_final.to_excel(writer, sheet_name="PubMed_clean", index=False)
    sheet = writer.sheets["PubMed_clean"]
    for idx, col in enumerate(df_final.columns, start=1):
        max_len = max(df_final[col].astype(str).apply(len).max(), len(col)) + 2
        sheet.column_dimensions[chr(64+idx)].width = min(max_len, 60)

print("- Excel exportado:", output_excel)

Artículos brutos: 1559
Tras filtrar año ≥2005: 1537
Tras filtrar tipo de publicación: 1442
Tras excluir 'conference' en journal: 1441
Artículos tras filtro clínico estricto: 1310
Filas finales: 1310
- CSV exportado: pubmed_clean_final.csv
- Excel exportado: pubmed_clean_final.xlsx


In [ ]:
df_final.head()

,PMID,TI,AB,JT,year,pub_type,AU,OT,institution,country,DOI,url
0,41538537,Global awareness and integration of immunother...,OBJECTIVE: Lung cancer is the leading cause of...,Revista da Associacao Medica Brasileira (1992),2026,article,"Ozkaya M, Yalcin NC, Guler A, Onder AH, Guzel HG",nan,University of Health Sciences,Turkey,10.1590/1806-9282.20250540,https://pubmed.ncbi.nlm.nih.gov/41538537/
1,41522164,Clinicopathological-CT model for predicting PD...,BACKGROUND: The expression of programmed death...,Journal of thoracic disease,2025,article,"Zhuo Y, Wang Q, Zhan Y, Yang S, Zhang H, Yang ...","Non-small cell lung cancer (NSCLC), computed t...",Department of Radiology,China,10.21037/jtd-2025-1439,https://pubmed.ncbi.nlm.nih.gov/41522164/
2,41522147,Feasibility and safety of outside the cage sub...,BACKGROUND: The utilization of robotic-assiste...,Journal of thoracic disease,2025,article,"Bulgarelli Maqueda L, Guimaraes Rocha Lima P, ...","Subcostal, lobectomy, non-intercostal, outside...",Department of Thoracic Surgery,Canada,10.21037/jtd-2025-1505,https://pubmed.ncbi.nlm.nih.gov/41522147/
3,41510384,Exploring perioperative treatment for non-smal...,BACKGROUND: For patients with resectable epide...,Translational lung cancer research,2025,article,"Zhou Y, Wei Z, Li M, Li J, Meng R, Wu F, Jiang...","Non-small cell lung cancer (NSCLC), cohort stu...",The Second Department of Thoracic Oncology,China,10.21037/tlcr-2025-962,https://pubmed.ncbi.nlm.nih.gov/41510384/
4,41510377,The effects of neoadjuvant sintilimab versus p...,BACKGROUND: The effectiveness of different imm...,Translational lung cancer research,2025,article,"Sun Y, Gao Z, Luo Z, Tong L, Zhang Y, Dong X, ...","Non-small cell lung cancer (NSCLC), effectiven...",Department of Thoracic Surgery,China,10.21037/tlcr-2025-aw-1256,https://pubmed.ncbi.nlm.nih.gov/41510377/


## Construcción del dataset maestro

Se unifican los datos de ClinicalTrials y Pubmed en un único dataset

In [ ]:
import pandas as pd
import numpy as np

# FUNCIÓN PARA MAPEAR TIPO DE DOCUMENTO PUBMED


def map_pub_type(pt):
   """
   Mapea el tipo de publicación a una categoría en español.
   """
   if pd.isna(pt):
      return "Artículo original"

   pt_low = pt.lower()

   if "systematic" in pt_low:
       return "Revisión sistemática"
   if "meta" in pt_low:
       return "Metaanálisis"
   if "review" in pt_low:
       return "Revisión"
   if "clinical trial" in pt_low:
       return "Ensayo clínico publicado"
   if "case" in pt_low:
       return "Caso clínico"
   if "editorial" in pt_low:
       return "Editorial"
   if "letter" in pt_low:
       return "Carta"

   return "Artículo original"


# TRANSFORMACIÓN PUBMED → FORMATO MAESTRO
# df_pubmed = dataframe final limpio de PubMed


def transform_pubmed(df_pubmed):
  """
  Transforma registros de PubMed a un formato tabular unificado.
  """

  df = df_pubmed.copy()

  df["source"] = "PubMed"
  df["id"] = df["PMID"]
  df["title"] = df["TI"]
  df["abstract_summary"] = df["AB"]

  # Ya tienes "year"
  df["countries"] = df["country"]
  df["institution"] = df["institution"]
  df["intervention"] = df["OT"]  # keywords
  df["url"] = df["url"]

  # Tipo de documento
  df["tipo_documento"] = df["pub_type"].apply(map_pub_type)

  # Marcadores clínicos aproximados
  df["is_IO"] = df["AB"].str.contains("immun|pd-|checkpoint|pdl1", case=False,
                                      na=False)
  df["is_surgery"] = df["AB"].str.contains(
      "resect|lobectomy|surgery|thoracotomy|vats|rats", case=False, na=False)
  df["is_peri"] = df["AB"].str.contains(
      "neoadjuvant|perioperative|preoperative", case=False, na=False)

  # Campo NLP combinado
  df["text_for_nlp"] = (
      df["TI"].fillna("") + " " +
      df["AB"].fillna("") + " " +
      df["OT"].fillna(""))

  return df[[
      "source","id","title","abstract_summary",
      "year","countries","institution","intervention",
      "is_IO","is_surgery","is_peri",
      "text_for_nlp","url","tipo_documento"]]



# TRANSFORMACIÓN CLINICALTRIALS → FORMATO MAESTRO
# df_final_trials = dataframe limpio final de ClinicalTrials


def transform_trials(df_trials):
  """
  Transforma ensayos de ClinicalTrials a un formato tabular unificado.
  """

  df = df_trials.copy()

  df["source"] = "ClinicalTrials"
  df["id"] = df["NCTId"]
  df["title"] = df["title"]
  df["abstract_summary"] = df.get("summary", None)

  # Año desde start_date
  df["year"] = df["start_date"].str[:4].astype(float)

  # Países ya limpios
  df["countries"] = df["countries"]

  # ClinicalTrials no ofrece institución fácilmente
  df["institution"] = None

  df["intervention"] = df["interventions"]

  df["url"] = "https://clinicaltrials.gov/study/" + df["NCTId"]

  # Marcadores clínicos ya vienen en el dataset
  # df["is_IO"], df["is_surgery"], df["is_peri"] ya existen

  df["tipo_documento"] = "Ensayo clínico"

  # Campo NLP
  df["text_for_nlp"] = (
      df["title"].fillna("") + " " +
      df["abstract_summary"].fillna("") + " " +
      df["interventions"].fillna("")
  )

  return df[[
      "source","id","title","abstract_summary",
      "year","countries","institution","intervention",
      "is_IO","is_surgery","is_peri",
      "text_for_nlp","url","tipo_documento"
  ]]


# CREAR EL DATASET MAESTRO

pub_master = transform_pubmed(df_final)
ct_master = transform_trials(df_final_trials)

df_master = pd.concat([pub_master, ct_master], ignore_index=True)

print("Tamaño dataset maestro:", df_master.shape)
df_master.head()


# EXPORTACIÓN

df_master.to_csv("master_dataset_pubmed_clinicaltrials.csv", index=False,
                 encoding="utf-8-sig")

with pd.ExcelWriter("master_dataset_pubmed_clinicaltrials.xlsx",
                    engine="openpyxl") as writer:
    df_master.to_excel(writer, sheet_name="master", index=False)

print("Dataset maestro exportado correctamente")

Tamaño dataset maestro: (1451, 14)
Dataset maestro exportado correctamente


In [ ]:
df_master.head()

,source,id,title,abstract_summary,year,countries,institution,intervention,is_IO,is_surgery,is_peri,text_for_nlp,url,tipo_documento
0,PubMed,41538537,Global awareness and integration of immunother...,OBJECTIVE: Lung cancer is the leading cause of...,2026.0,Turkey,University of Health Sciences,nan,True,True,True,Global awareness and integration of immunother...,https://pubmed.ncbi.nlm.nih.gov/41538537/,Artículo original
1,PubMed,41522164,Clinicopathological-CT model for predicting PD...,BACKGROUND: The expression of programmed death...,2025.0,China,Department of Radiology,"Non-small cell lung cancer (NSCLC), computed t...",True,True,True,Clinicopathological-CT model for predicting PD...,https://pubmed.ncbi.nlm.nih.gov/41522164/,Artículo original
2,PubMed,41522147,Feasibility and safety of outside the cage sub...,BACKGROUND: The utilization of robotic-assiste...,2025.0,Canada,Department of Thoracic Surgery,"Subcostal, lobectomy, non-intercostal, outside...",True,True,True,Feasibility and safety of outside the cage sub...,https://pubmed.ncbi.nlm.nih.gov/41522147/,Artículo original
3,PubMed,41510384,Exploring perioperative treatment for non-smal...,BACKGROUND: For patients with resectable epide...,2025.0,China,The Second Department of Thoracic Oncology,"Non-small cell lung cancer (NSCLC), cohort stu...",True,True,True,Exploring perioperative treatment for non-smal...,https://pubmed.ncbi.nlm.nih.gov/41510384/,Artículo original
4,PubMed,41510377,The effects of neoadjuvant sintilimab versus p...,BACKGROUND: The effectiveness of different imm...,2025.0,China,Department of Thoracic Surgery,"Non-small cell lung cancer (NSCLC), effectiven...",True,True,True,The effects of neoadjuvant sintilimab versus p...,https://pubmed.ncbi.nlm.nih.gov/41510377/,Artículo original
